1. preprocessing in FIJI: cut out single colonies, increase contrast, substrate BG (ball 50), threshold and create mask, mask apply on BG-sub (nbg) [not used]
2. Ilastik label pixels, segment. Manual correction with Fiji
3. Contour analysis with cv2
4. TrackMate, map contours and tracks between TrackMate and cv2

Some old code on cell-tracking directly from cv2 contour, skeleton analysis, 

Note: 
* Colony-level analysis (area growth rate, MTMT area, density) done with "Area-analysis"
* Plot check on video moved to "RatePlot" -cellID

In [1]:
# import the essential packages 
import numpy as np   # for numerics
import glob   #for parsing directories and files
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt  #for plotting
import random
import cv2
import os
from matplotlib import cm
import json
import pickle
import pandas as pd


np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

In [6]:
"""
20250303 rewritten by ChatGPT
cv2 contour analysis, result given in DataFrame
20250309 revise, "bounding box" + "fit elipse" 
Note: "skeleton analysis" is done by another library, if needed, may need to match contour again

---Input---
Ilastik segmented, binary image series. 
Cells must be separated (# check by plot contour and TrackMate).

---Output---
Pandas DataFrame: 
["frame", "spot_ID", "x", "y", "area", "solidity", "rect_length", "rect_aspect_ratio", "rect_angle", "ellipse_long_axis", "ellipse_theta"]
_Note:_ when read in TrackMate spots.csv, usecols =["ID", "POSITION_X","POSITION_Y", "FRAME", "ELLIPSE_MAJOR","ELLIPSE_THETA", "SOLIDITY"] 
"""
rplcpath ='ForPub/150-LBLMagar/250717-150-2-ana/00955'
# rplcpath ='ForPub/150-LBLMagar/240228-150-1-ana/00321-2'
head, rplcIdx = os.path.split(rplcpath)
tmpath = rplcpath + '/trackmate'

segName = rplcIdx + '-BG_Simple Segmentation.tiff' 
fpath = os.path.join(rplcpath, segName)
dataFName = rplcIdx + '-spots_cv2.csv'
dataFpath = os.path.join(tmpath, dataFName)

img_segs = []
ret, img_segs = cv2.imreadmulti(mats=img_segs, filename=fpath, 
#                                 start=0, count=81,
                                flags=cv2.IMREAD_GRAYSCALE)
data_list = [] 
for i in range(len(img_segs)):
    img_seg = img_segs[i]
    contours, hierarchy = cv2.findContours(img_seg, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
 
    for j, ct in enumerate(contours):
        area = cv2.contourArea(ct)
        if area > 50:  # Size range for cells
            M = cv2.moments(ct)
            if M['m00'] == 0:
                continue  # Avoid division by zero           
            # Compute centroid
            cntx = M['m10'] / M['m00']
            cnty = M['m01'] / M['m00']
            
            # Get minimum area bounding rectangle
            rect = cv2.minAreaRect(ct)
            width, height = rect[1]  # (w, h) of the rotated bounding box
            angle_rect = rect[2]           
            # Ensure width is always the smaller side and height is the longer side
            length_rect = max(width, height)
            aspect_ratio = length_rect / min(width, height)

            ellipse = cv2.fitEllipse(ct)  # (center_x, center_y), (major_axis, minor_axis), angle
            major_axis, minor_axis = ellipse[1]  # note: from test, it's (minor, major)
            length_e = max(major_axis, minor_axis)
            angle_e = ellipse[2]

             # Compute Convex Hull & its Area
            hull = cv2.convexHull(ct)
            hull_area = cv2.contourArea(hull)       
            # Compute Solidity
            solidity = area / hull_area if hull_area > 0 else 0
    
            data_list.append([i, j, cntx, cnty, area, solidity, length_rect, aspect_ratio, angle_rect, length_e, angle_e])

# Create DataFrame
columns = ["frame", "spot_ID", "x", "y", "area", "solidity", 
           "rect_length", "rect_aspect_ratio", "rect_angle", "ellipse_long_axis", "ellipse_theta"]
df = pd.DataFrame(data_list, columns=columns)

# Save DataFrame as a CSV file
df.to_csv(dataFpath, index=False)            

In [7]:
"""
Match data between contour DataFrame and tracks from TrackMate
----input----
trackmate/-BG_spots.csv, -spots_cv2.csv

----output----
-spots_cv2-tmmap.csv
"""
from scipy.spatial import KDTree
exppath = 'ForPub/150-LBLMagar/250717-150-2-ana/'
# exppath = 'ForPub/075-LBLMagar/250117-075-2-ana/'
rplcpathList = [f.path for f in os.scandir(exppath) if f.is_dir() and '00955' in f.name]
for rplcpath in sorted(rplcpathList):
    head, rplcIdx = os.path.split(rplcpath) 

    tmpath = rplcpath + '/trackmate'
    spotsName =  rplcIdx + "-BG_spots.csv" 
    spotspath = os.path.join(tmpath, spotsName)
    dataFName = rplcIdx + '-spots_cv2.csv'
    dataFpath = os.path.join(tmpath, dataFName)
    outName = rplcIdx + '-spots_cv2_tmmap.csv'
    outpath = os.path.join(tmpath, outName)

    # Load TrackMate and opencv results
    trackmate_df = pd.read_csv(spotspath, skiprows=[1, 2, 3], header=0, usecols =["ID", "POSITION_X","POSITION_Y", "FRAME"])  
    opencv_df = pd.read_csv(dataFpath)

    # Create a dictionary of KDTree per frame using TrackMate spots
    frame_trees = {}
    for frame, group in trackmate_df.groupby("FRAME"):
        # Build KDTree for TrackMate spots in this frame
        frame_trees[frame] = (KDTree(group[["POSITION_X", "POSITION_Y"]]), group)


    # Initialize the new column in OpenCV DataFrame
    opencv_df["tm_spot_ID"] = None  
    search_radius = 10

    # Map opencv contour to the nearest trackmate spot
    for i, row in opencv_df.iterrows():
        frame, x, y = row['frame'], row['x'], row['y']
        if frame in frame_trees:
            tree, frame_data = frame_trees[frame]
            dist, idx = tree.query([x, y], distance_upper_bound=search_radius)

            # Only assign a match if the nearest TrackMate spot is within the threshold
            if dist < search_radius:
                trackmate_spot = frame_data.iloc[idx]
                opencv_df.at[i, "tm_spot_ID"] = int(trackmate_spot["ID"])

    # Save mapping results
    opencv_df.to_csv(outpath, index=False)


In [5]:
print(opencv_df.head())


   frame  spot_ID           x           y    area  solidity  rect_length  \
0      0        0  410.980234  395.268793  1113.0  0.918317    71.059067   
1      1        0  405.441001  389.843882  1145.5  0.916400    73.095490   
2      2        0  404.394541  387.734769  1184.5  0.922508    72.622131   
3      3        0  403.521800  386.046525  1196.5  0.909886    74.651062   
4      4        0  402.726996  384.930262  1204.5  0.907003    76.394119   

   rect_aspect_ratio  rect_angle  ellipse_long_axis  ellipse_theta tm_spot_ID  
0           3.729508   38.659805          81.430725     128.840820       8896  
1           3.795306   38.884495          83.303757     129.393982       8903  
2           3.608054   38.418053          84.139160     129.189056       8892  
3           3.793651   38.659809          86.977219     128.899750       8895  
4           3.854569   37.746803          90.296951     129.144623       8893  


In [8]:
"""
Reconstruct cell tracks from "edge.csv" given by TrackMate and opencv analysis
only cells roots back to begining

----input----
"_edges.csv" : [SPOT_SOURCE_ID,SPOT_TARGET_ID,EDGE_TIME]
'-spots_cv2_tmmap.csv': ["frame", "spot_ID", "x", "y", "area", "solidity", 
"rect_length", "rect_aspect_ratio", "rect_angle", "ellipse_long_axis", "ellipse_theta", "tm_spot_ID"]

---Output---
Tracks DataFrame
 ["cell_ID", "cell_parent_ID", "cell_daughter1_ID", "cell_daughter2_ID", "lineage", "generation", "tm_spot_ID", "frame", 
 "x", "y", "area", "solidity", "rect_length", "rect_aspect_ratio", "rect_angle", "ellipse_long_axis", "ellipse_theta"]

 _Note:_ not all contours has tm_spot_ID
"""
# exppath = 'ForPub/075-LBLMagar/241219-075-ana/'
# rplcpathList = [f.path for f in os.scandir(exppath) if f.is_dir() and '00' in f.name]
# rplcpathList = ['ForPub/150-LBLMagar/240228-150-1-ana/00321-2']

for rplcpath in sorted(rplcpathList):
    head, rplcIdx = os.path.split(rplcpath) 
    
    # rplcpath ='ForPub/600-LBLMagar/00798-2'
    # head, rplcIdx = os.path.split(rplcpath)
    tmpath = rplcpath + '/trackmate'
    edgesName =  rplcIdx + "-BG_edges.csv" 
    edgespath = os.path.join(tmpath, edgesName)
    spotsName =  rplcIdx + '-spots_cv2_tmmap.csv'
    spotspath = os.path.join(tmpath, spotsName)
    tracksName = rplcIdx + "-tracks-cv2-raw.csv" 
    trackspath = os.path.join(rplcpath, tracksName)


    edges_df = pd.read_csv(edgespath, skiprows=[1, 2, 3], header=0, 
                           usecols =["SPOT_SOURCE_ID", "SPOT_TARGET_ID", "EDGE_TIME"] )
    spots_df = pd.read_csv(spotspath)

    # Ensure data types are correct
    edges_df["SPOT_SOURCE_ID"] = edges_df["SPOT_SOURCE_ID"].astype(int)
    edges_df["SPOT_TARGET_ID"] = edges_df["SPOT_TARGET_ID"].astype(int)
    edges_df["EDGE_TIME"] = edges_df["EDGE_TIME"].astype(float)  # Ensure numeric
    # spots_df["tm_spot_ID"] = spots_df["tm_spot_ID"].astype(int) 
    # spots_df["frame"] = spots_df["frame"].astype(int)

    # Sort dataframes to process tracks sequentially
    edges_df = edges_df.sort_values(by="EDGE_TIME").reset_index(drop=True)
    spots_df = spots_df.sort_values(by="frame").reset_index(drop=True)

    # Initialize tracking structures
    cell_tracks = {}  # {cell_ID: [list of spot_IDs]}
    cell_map = {}  # {spot_ID: cell_ID}
    parent_map = {}  # {cell_ID -> parent_ID}  
    daughter_map = {}  # {cell_ID: [daughter1_ID, daughter2_ID]}

    # Identify root cells (cells in the first frame, can handle multiple cells)
    first_frame = edges_df["EDGE_TIME"].min() - 0.5
    root_edges = edges_df[edges_df["EDGE_TIME"] == first_frame+0.5]   

    cell_counter = 0
    for _, row in root_edges.iterrows():
        source_spot = row["SPOT_SOURCE_ID"]
        target_spot = row["SPOT_TARGET_ID"]

        # Assign cell_IDs
        cell_map[source_spot] = cell_counter
        cell_map[target_spot] = cell_counter
        parent_map[cell_counter] = None  # Root cells have no parent
        cell_tracks[cell_counter] = [source_spot, target_spot]

        cell_counter += 1

    # Process edges to extend tracks and detect division
    for _, row in edges_df.iterrows():
        # Check for division
        source_spot = row["SPOT_SOURCE_ID"]
        target_spot = row["SPOT_TARGET_ID"]
        frame_id = int(row["EDGE_TIME"] - 0.5)  # Convert EDGE_TIME to integer frame    

        children = edges_df[edges_df["SPOT_SOURCE_ID"] == source_spot]["SPOT_TARGET_ID"].tolist() # list of targetID  
        if len(children) == 1:  
            if source_spot in cell_map and not(target_spot in cell_map):
                cell_id = cell_map[source_spot]  #cell_id, also used for division
                # Extend track by linking TARGET to SOURCE in the next frame
                cell_map[target_spot] = cell_id
                cell_tracks[cell_id].append(target_spot)

        elif len(children) == 2: # handle two children at the same time, should avoid repeat
            if source_spot in cell_map and not(target_spot in cell_map):
                parent_id = cell_map[source_spot]
    #             print(f"Division detected at frame {frame_id} for cell {parent_id} {source_spot} → daughters: {children}")

                daughter1_id = cell_counter
                daughter2_id = cell_counter + 1

                cell_map[children[0]] = daughter1_id
                cell_map[children[1]] = daughter2_id
                parent_map[daughter1_id] = parent_id
                parent_map[daughter2_id] = parent_id
                daughter_map[parent_id] = [daughter1_id, daughter2_id]

                cell_tracks[daughter1_id] = [children[0]]
                cell_tracks[daughter2_id] = [children[1]]
                cell_counter += 2
        else:
            print(source_spot, f"abnormal")


    #build lineage from parent_map 
    lineage_map = {}  # {cell_ID -> whole lineage from root cell} 
    for cell_id in cell_tracks.keys():
        lineage_map[int(cell_id)] = [cell_id]
    for cell_id in lineage_map.keys():
        parent_id = parent_map[cell_id]
        if parent_id is not None:  # If not a root cell
            lineage_map[cell_id] = lineage_map[int(parent_id)] + [cell_id]
        else:
            lineage_map[cell_id] = [cell_id] 
    # Print tracking results for debugging
    # print("\nFinal cell tracks:")
    # for cell_id, track in cell_tracks.items():
    #     print(f"Cell {cell_id}: {track}")


    # Convert track data into DataFrame format
     # ["cell_ID", "cell_parent_ID", "cell_daughter1_ID", "cell_daughter2_ID", "lineage", "generation", "tm_spot_ID", "frame", 
     # "x", "y", "area", "solidity", "rect_length", "rect_aspect_ratio", "rect_angle", "ellipse_long_axis", "ellipse_theta"]
    lineage_data = []
    for cell_id, spot_list in cell_tracks.items():
       for spot in spot_list:
            # Select relevant row(s) for the current spot_ID
            spot_data = spots_df.loc[spots_df["tm_spot_ID"] == spot, 
                ["frame", "x", "y", "area", "solidity", "rect_length", 
                 "rect_aspect_ratio", "rect_angle", "ellipse_long_axis", "ellipse_theta"]]

            # Only proceed if there is a matching row
            if not spot_data.empty:  
                lineage_data.append({
                    "cell_ID": cell_id,
                    "cell_parent_ID": parent_map.get(cell_id),
                    "cell_daughter1_ID": daughter_map.get(cell_id, [None, None])[0],
                    "cell_daughter2_ID": daughter_map.get(cell_id, [None, None])[1],
                    "lineage": lineage_map[cell_id],
                    "generation": len(lineage_map[cell_id])-1,
                    "spot_ID": spot,
                    **spot_data.iloc[0].to_dict().copy()  # Convert row to dictionary safely
                })
            else:
                print(f"Warning: {rplcIdx} spot_ID {spot} not found in spots_df.") 

    # Convert to DataFrame
    lineage_df = pd.DataFrame(lineage_data)

    # Display results
    # print(lineage_df.head())

    # Save to CSV
    lineage_df.to_csv(trackspath, index=False)

In [32]:
"""
250306 modified from ChatGPT script

Reconstruct cell tracks from "edge.csv" given by TrackMate
The output datafile 
----input----
"_edges.csv" : [SPOT_SOURCE_ID,SPOT_TARGET_ID,EDGE_TIME]
"_spots.csv": [ID, POSITION_X,POSITION_Y, FRAME, ELLIPSE_MAJOR,ELLIPSE_THETA, SOLIDITY]

---Output---
Tracks DataFrame
 ["cell_ID", "cell_parent_ID", "cell_daughter1_ID", "cell_daughter2_ID", "lineage", "generation"
 "spot_ID", "frame", "x", "y", "ellipse_long_axis", "ellipse_theta", "solidity" ]
 
Note:
in TrackMate spot analysis, "ELLIPSE_MAJOR" may not be the long axis
"""
rplcpath ='ForPub/075-LBLMagar/00823'
head, rplcIdx = os.path.split(rplcpath)
tmpath = rplcpath + '/trackmate'

edgesName =  rplcIdx + "-BG_edges.csv" 
edgespath = os.path.join(tmpath, edgesName)
spotsName =  rplcIdx + "-BG_spots.csv" 
spotspath = os.path.join(tmpath, spotsName)
tracksName = rplcIdx + "-tracks-tm.csv" 
trackspath = os.path.join(rplcpath, tracksName)


edges_df = pd.read_csv(edgespath, skiprows=[1, 2, 3], header=0, 
                       usecols =["SPOT_SOURCE_ID", "SPOT_TARGET_ID", "EDGE_TIME"] )
spots_df = pd.read_csv(spotspath, skiprows=[1, 2, 3], header=0, 
                       usecols =["ID", "POSITION_X","POSITION_Y", "FRAME", "ELLIPSE_MAJOR","ELLIPSE_MINOR","ELLIPSE_THETA", "SOLIDITY"] )

# Ensure data types are correct
edges_df["SPOT_SOURCE_ID"] = edges_df["SPOT_SOURCE_ID"].astype(int)
edges_df["SPOT_TARGET_ID"] = edges_df["SPOT_TARGET_ID"].astype(int)
edges_df["EDGE_TIME"] = edges_df["EDGE_TIME"].astype(float)  # Ensure numeric
spots_df["ID"] = spots_df["ID"].astype(int)
spots_df["FRAME"] = spots_df["FRAME"].astype(int)

# Sort dataframes to process tracks sequentially
edges_df = edges_df.sort_values(by="EDGE_TIME").reset_index(drop=True)
spots_df = spots_df.sort_values(by="FRAME").reset_index(drop=True)

# Initialize tracking structures
cell_tracks = {}  # {cell_ID: [list of spot_IDs]}
cell_map = {}  # {spot_ID: cell_ID}
parent_map = {}  # {cell_ID -> parent_ID}  
daughter_map = {}  # {cell_ID: [daughter1_ID, daughter2_ID]}
        
# Identify root cells (cells in the first frame, can handle multiple cells)
first_frame = edges_df["EDGE_TIME"].min() - 0.5
root_edges = edges_df[edges_df["EDGE_TIME"] == first_frame+0.5]   

cell_counter = 0
for _, row in root_edges.iterrows():
    source_spot = row["SPOT_SOURCE_ID"]
    target_spot = row["SPOT_TARGET_ID"]

    # Assign cell_IDs
    cell_map[source_spot] = cell_counter
    cell_map[target_spot] = cell_counter
    parent_map[cell_counter] = None  # Root cells have no parent
    cell_tracks[cell_counter] = [source_spot, target_spot]

    cell_counter += 1

# Process edges to extend tracks and detect division
for _, row in edges_df.iterrows():
    # Check for division
    source_spot = row["SPOT_SOURCE_ID"]
    target_spot = row["SPOT_TARGET_ID"]
    frame_id = int(row["EDGE_TIME"] - 0.5)  # Convert EDGE_TIME to integer frame    
    
    children = edges_df[edges_df["SPOT_SOURCE_ID"] == source_spot]["SPOT_TARGET_ID"].tolist() # list of targetID  
    if len(children) == 1:  
        if source_spot in cell_map and not(target_spot in cell_map):
            cell_id = cell_map[source_spot]  #cell_id, also used for division
            # Extend track by linking TARGET to SOURCE in the next frame
            cell_map[target_spot] = cell_id
            cell_tracks[cell_id].append(target_spot)

    elif len(children) == 2: # handle two children at the same time, should avoid repeat
        if source_spot in cell_map and not(target_spot in cell_map):
            parent_id = cell_map[source_spot]
#             print(f"Division detected at frame {frame_id} for cell {parent_id} {source_spot} → daughters: {children}")

            daughter1_id = cell_counter
            daughter2_id = cell_counter + 1

            cell_map[children[0]] = daughter1_id
            cell_map[children[1]] = daughter2_id
            parent_map[daughter1_id] = parent_id
            parent_map[daughter2_id] = parent_id
            daughter_map[parent_id] = [daughter1_id, daughter2_id]

            cell_tracks[daughter1_id] = [children[0]]
            cell_tracks[daughter2_id] = [children[1]]
            cell_counter += 2
    else:
        print(source_spot, f"abnormal")


#build lineage from parent_map 
lineage_map = {}  # {cell_ID -> whole lineage from root cell} 
for cell_id in cell_tracks.keys():
    lineage_map[int(cell_id)] = [cell_id]
for cell_id in lineage_map.keys():
    parent_id = parent_map[cell_id]
    if parent_id is not None:  # If not a root cell
        lineage_map[cell_id] = lineage_map[int(parent_id)] + [cell_id]
    else:
        lineage_map[cell_id] = [cell_id] 
# Print tracking results for debugging
# print("\nFinal cell tracks:")
# for cell_id, track in cell_tracks.items():
#     print(f"Cell {cell_id}: {track}")
            
            
# Convert track data into DataFrame format
# ["cell_ID", "cell_parent_ID", "cell_daughter1_ID", "cell_daughter2_ID", "lineage",
#  "spot_ID", "frame", "x", "y", "ellipse_long_axis", "ellipse_theta", "solidity" ]
lineage_data = []
for cell_id, spot_list in cell_tracks.items():
    for spot in spot_list:
#         frame = edges_df.loc[edges_df["SPOT_SOURCE_ID"] == spot, "EDGE_TIME"].min()
#         frame_id = frame - 0.5 if not pd.isna(frame) else None #TODO: add last
        lineage_data.append({
            "cell_ID": cell_id,
            "cell_parent_ID": parent_map[cell_id],
            "cell_daughter1_ID": daughter_map[cell_id][0] if cell_id in daughter_map else None,
            "cell_daughter2_ID": daughter_map[cell_id][1] if cell_id in daughter_map else None,
            "lineage": lineage_map[cell_id],
            "generation": len(lineage_map[cell_id])-1,
            "spot_ID": spot,
            "frame": spots_df.loc[spots_df["ID"] == spot, "FRAME"].iloc[0],
            "x": spots_df.loc[spots_df["ID"] == spot, "POSITION_X"].iloc[0],
            "y": spots_df.loc[spots_df["ID"] == spot, "POSITION_Y"].iloc[0],
            "ellipse_long_axis": spots_df.loc[spots_df["ID"] == spot, "ELLIPSE_MAJOR"].iloc[0],  # need revise
            "ellipse_theta": spots_df.loc[spots_df["ID"] == spot, "ELLIPSE_THETA"].iloc[0],
            "solidity": spots_df.loc[spots_df["ID"] == spot, "SOLIDITY"].iloc[0]
        })
# usecols =["ID", "POSITION_X","POSITION_Y", "FRAME", "ELLIPSE_MAJOR","ELLIPSE_THETA", "SOLIDITY"] 
    
# Convert to DataFrame
lineage_df = pd.DataFrame(lineage_data)

# Display results
# print(lineage_df.head())

# Save to CSV
lineage_df.to_csv(trackspath, index=False)

In [26]:
lineage_map = {0: [0], 1: [1], 2: [2], 3: [3], 4: [4]}
parent_map = {0: np.nan, 1: 0, 2: 0, 3: 1, 4: 1}

for cell_id in lineage_map.keys():
    parent_id = parent_map[cell_id]
    if not np.isnan(parent_id):  # If not a root cell
        lineage_map[cell_id] = lineage_map[int(parent_id)] + [cell_id]
    else:
        lineage_map[cell_id] = [cell_id] 
print(lineage_map)

{0: [0], 1: [0, 1], 2: [0, 2], 3: [0, 1, 3], 4: [0, 1, 4]}


In [36]:
"""
1st version, cv2 contour analysis, result given in Dictionary of numpy array

-----Input----
Ilastik segmented, binary image series. Cells must be separated.
-----Output-----
Dictionary: [frameIdx]: nparray #  cellIdx, cntx, cnty, 3rect0, 4rect1, 5area, 6aspect ratio(AR), 7length, 8angle
"""

fpath = os.path.join(rplcpath, segName)
cnpath = os.path.join(rplcpath, dataDicName)
print(cnpath)

img_segs = []
ret,img_segs = cv2.imreadmulti(mats=img_segs, 
                               filename=fpath, flags=cv2.IMREAD_GRAYSCALE)
print(len(img_segs))
dataDic = {} # empty dictionary [frameIdx]: fData
for i in range(len(img_segs)):
    img_seg = img_segs[i]
#     # convert 1 2 label to 0 and 255
#     one = np.array(1)
#     img_seg = np.subtract(img_seg, one)
#     img_seg = img_seg *255 

#     # agar movies are black on white
#     inv = np.array(255)
#     img_seg = np.subtract(inv, img_seg)

    contours, hierarchy = cv2.findContours(img_seg, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    fData = np.array([]).reshape(0,9)  # cellIdx, cntx, cnty, 3rect0, 4rect1, 5area, 6aspect ratio(AR), 7length, 8angle
    for j,ct in enumerate(contours):
        area = cv2.contourArea(ct)
        cellData = np.zeros(9)       
        if area > 100 and area < 5000:   # size range for cells 
            cellData[0] = j
            cellData[5] = area
            M = cv2.moments(ct)
            cellData[1] = M['m10']/M['m00']  # cntx    
            cellData[2] = M['m01']/M['m00']  # cnty
            
            peri = cv2.arcLength(ct,True)
#             approx = cv2.approxPolyDP(cnt,0.00009*peri,True)
            # bounding box to get length, width, AR, angle
            rect = cv2.minAreaRect(ct)
#             box = cv2.boxPoints(rect)
#             box = np.int0(box)
            # Retrieve the key parameters of the rotated bounding box
#             center = (int(rect[0][0]),int(rect[0][1])) 
            cellData[3] = rect[1][0]           
            cellData[4] = rect[1][1]   
            if cellData[3] >= cellData[4]:
                cellData[6] = cellData[3] / cellData[4]  # length / width
                cellData[7] = cellData[3]
            else:
                cellData[6] = cellData[4] / cellData[3] 
                cellData[7] = cellData[4]
            cellData[8] = rect[2]   # angle
            fData = np.vstack((fData, cellData))
            
    dataDic[str(i)] = fData
#     print(len(contours))
#     print(i, j, fData)

with open(cnpath, 'wb') as f:
    pickle.dump(dataDic, f)
f.close()


ForPub/300-LBLMagar/00766\00766-dataDic.pkl
145


In [39]:
"""
old Trajectory (cell lineage) analysis for agar, with ellipse 
----
Input:
    Dictionary: [frameIdx]: nparray # cellIdx, cntx, cnty, length, width, aspect ratio(AR), area, angle
Output:
    1. array for split events: # fIdx, Mother cell Idx, M length, Daughter cell Idx1, D1 length, D2 Idx, D2 length
    2. Trajectories: # fIdx, cellIdx, cntx, cnty, length, area, angle
----
Find all the cells on the next frame overlap with the cell on the previous frame, cnt distance < 0.4(?) cell length
(images aligned in 1st step)
0 cell: close traj; 1 cell: traj+1; 2 cells: start 2 new trajs, record a division.
Not include: (1) find >2 cells within threshold

---20240205 update---
ellipse 2a, 2b, angle following rect, cnt follows contour cnt,
Find daughter cells based on if new cnt is inside elipse

---20240421 update---
include traj index into div file
---20250304 revised by ChatGPT, bug fixed for counting trajs multiple times---

"""
# exppath = 'glass-LBLMagar-003/240620-003LBLMagar-ana/'  #240529, 240530, 240612, 240619, 240620
# exppath = 'glass-LBLMagar-0075/240618-0075LBLMagar-ana/'  # 240530, 240613, 240618
# exppath = 'glass-LBLMagar-010/240613-010LBLMagar-ana/' # 240613, 240618
# exppath = 'glass-LBLMagar-015/240228-agar-GF-ana/'  #240228 240307
# exppath = 'LB-glass-coverslip/240229-glass-GF-ana/'
# exppath = 'glass-LBLMagar/Elongations/240307/'  #   240307 231006-60x

# rplcpathList = [f.path for f in os.scandir(exppath) if f.is_dir()]

# rplcpath = rplcpathList[0]
# rplcpath = 'glass-LBLMagar-015/240307-agar-GF-ana/00392-3'
# head, rplcIdx = os.path.split(rplcpath) 

# nbgName = rplcIdx + '-BG.tif'
# # prepName = dataIdx + '-BG-prep.tif'
# segName = rplcIdx + '-BG_Simple Segmentation.tiff' 
# dataDicName = rplcIdx + '-dataDic.pkl'
# divName = rplcIdx + '-Div.txt'
# trajDicName = rplcIdx + '-trajDic.pkl'

# fpath = os.path.join(rplcpath, segName)
cnpath = os.path.join(rplcpath, dataDicName)
trajpath = os.path.join(rplcpath, trajDicName)
divpath =  os.path.join(rplcpath, divName)

# dist = np.linalg.norm(p1-p2)  
# 2a = rect[1][0], 2b = rect[1][1], angle = rect[2], alpha = angle (in rad)
def inEllipse(x, y, a, b, m, n, alpha):
    term1 = ((x-m)*np.cos(alpha) + (y-n)*np.sin(alpha))**2 /(a**2)
    term2 = (-(x-m)*np.sin(alpha) + (y-n)*np.cos(alpha))**2 /(b**2)
    if term1 + term2 < 1:
        return True
    else:
        return False

with open(cnpath, 'rb') as f:
    dataDic = pickle.load(f)
trajDic = {}
division = np.array([]).reshape(0,7)
Header = '# 0fIdx, 1Mother cell Idx, 2M length, 3Daughter cell Idx1, 4D1 length, 5Daughter cell Idx2, 6D2 length'

# start trajs with all cells in frame#0
fData0 = dataDic[str(0)]  # cellIdx, cntx, cnty, 3rect0, 4rect1, 5area, 6aspect ratio(AR), 7length, 8angle
for i in range(fData0.shape[0]):
    traj = np.zeros([1,10]) # 0fIdx, 1cellIdx, 2cntx, 3cnty, 4rect0, 5rect1, 6area, 7aspect ratio(AR), 8length, 9angle
#     traj[0] = 0
    traj[0, 1:] = fData0[i, 0:]
    trajDic[str(i)] = traj
    
# Process frames sequentially
trajExKeys = list(trajDic.keys())  # Keys for ongoing trajectories

for k in range(1, len(dataDic.keys())):  # Read frames in order
    fData = dataDic[str(k)]  # Read current frame
    ttrajExKeys = trajExKeys.copy()
    used_cells = set()  # Track used cell indices in the new frame

    for j in trajExKeys:  # Loop through existing trajectories
        traj = trajDic[j]
        endData = traj[-1, :]  # Last known cell state

        # Extract ellipse parameters from the last frame
        cntX0, cntY0 = endData[2], endData[3]
        aa, bb = endData[4] *0.6, endData[5] *0.6  # Semi-major and semi-minor axes 250304 relax to 0.7
        aalpha = np.radians(endData[-1])  # Convert angle to radians
        dCounter = 0  # Track daughter cells
        daughter_cells = []

        for i in range(fData.shape[0]):  # Loop through new frame cells
            if i in used_cells:
                continue  # Skip cells already assigned to a trajectory

            cntX1, cntY1 = fData[i, 1], fData[i, 2]  # New cell center

            if inEllipse(cntX1, cntY1, aa, bb, cntX0, cntY0, aalpha):
                dCounter += 1
                daughter_cells.append(i)

                if dCounter == 1:  # First match -> extend trajectory
                    traj1 = np.zeros([1, 10])
                    traj1[0, 0] = k  # Frame index
                    traj1[0, 1:] = fData[i, :]
                    traj = np.vstack((traj, traj1))  # Extend trajectory
                    used_cells.add(i)  # Mark cell as used

                elif dCounter == 2:  # Second match -> division event
                    # Record division event
                    nDiv = np.zeros(7)
                    nDiv[0] = k - 1  # Frame of mother cell
                    nDiv[1] = endData[1]  # Mother cell ID
                    nDiv[2] = endData[-2]  # Mother cell length
                    nDiv[3] = fData[daughter_cells[0], 0]  # Daughter 1 ID
                    nDiv[4] = fData[daughter_cells[0], -2]  # Daughter 1 length
                    nDiv[5] = fData[daughter_cells[1], 0]  # Daughter 2 ID
                    nDiv[6] = fData[daughter_cells[1], -2]  # Daughter 2 length
                    division = np.vstack((division, nDiv))

                    # Close old trajectory
                    ttrajExKeys.remove(j)

                    # Start new trajectories for daughter cells
                    for dIdx in daughter_cells:
                        new_key = str(len(trajDic))  # Generate new trajectory key
                        new_traj = np.zeros([1, 10])
                        new_traj[0, 0] = k  # Frame index
                        new_traj[0, 1:] = fData[dIdx, :]
                        trajDic[new_key] = new_traj
                        ttrajExKeys.append(new_key)
                        used_cells.add(dIdx)  # Mark cell as used

                    break  # Exit loop once division is recorded

        if dCounter == 0:  # If no cell found, remove trajectory from active list
            ttrajExKeys.remove(j)
        
        trajDic[j] = traj  # Update trajectory dictionary

    trajExKeys = ttrajExKeys.copy()  # Update active trajectory keys
    
with open(trajpath, 'wb') as f:
    pickle.dump(trajDic, f)
f.close()
np.savetxt(divpath, division, fmt='%d', header=Header)

In [49]:
"""track plot check
write traj ID on to segmented tif
"""
from PIL import Image

with open(trajpath, 'rb') as f:
    trajDic = pickle.load(f)  # 0fIdx, 1cellIdx, 2cntx, 3cnty, 4rect0, 5rect1, 6area, 7aspect ratio(AR), 8length, 9angle
    
fpath = os.path.join(rplcpath, segName)
outName = rplcIdx + '-trajCheck.tif'
outpath = os.path.join(rplcpath, outName)
frames = []
frames_processed = []

font                   = cv2.FONT_HERSHEY_SIMPLEX
fontScale              = 0.7
fontColor              = (255,0,0)
thickness              = 2
lineType               = 2

ret,frames = cv2.imreadmulti(mats=frames, filename=fpath, 
#                               start=0,
#                               count=20,                              
                               flags=cv2.IMREAD_COLOR)
for frame_index, frame in enumerate(frames):
    # Convert the frame to RGB format if it's grayscale (required by PIL)
#     if len(frame.shape) == 2:  # grayscale
#         frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)
    for trajIdx, traj in trajDic.items():
        fIdxS = traj[0, 0]  # traj starting frm
        fIdxE = traj[-1, 0]  # traj starting frm
        if fIdxS <= frame_index  <= fIdxE: 
            traj_point = traj[(traj == float(frame_index))[:, 0]]
            cv2.putText(frame, trajIdx, (int(traj_point[0, 2]-7), int(traj_point[0, 3]-10)), font, fontScale, fontColor, thickness)
            
    frames_processed.append(Image.fromarray(frame))

# Save the processed frames as a TIFF stack
frames_processed[0].save(outpath, save_all=True, append_images=frames_processed[1:], compression="tiff_deflate")

In [ ]:
"""sample code for skeleton analysis"""


import cv2
import numpy as np
from skimage.morphology import skeletonize
from skimage.graph import route_through_array
from scipy.spatial.distance import pdist, squareform

# Load binary image (already binarized)
binary = cv2.imread("banana_shape.png", cv2.IMREAD_GRAYSCALE)

# Find contours
contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create a color image to visualize results
output_image = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)

for i, ct in enumerate(contours):
    if len(ct) < 5:
        continue  # Need at least 5 points for ellipse fitting

    # (1) Fit an ellipse
    ellipse = cv2.fitEllipse(ct)  # (center_x, center_y), (major_axis, minor_axis), angle
    cv2.ellipse(output_image, ellipse, (0, 255, 0), 2)  # Draw the ellipse in green

    # Create an empty mask for the current contour
    contour_mask = np.zeros_like(binary)
    cv2.drawContours(contour_mask, [ct], -1, 255, thickness=cv2.FILLED)

    # (2) Skeletonize the filled contour
    skeleton = skeletonize(contour_mask // 255)  # Convert 255 -> 1 before skeletonizing

    # Get skeleton pixel coordinates
    skel_coords = np.column_stack(np.where(skeleton > 0))  # Get (row, col) positions

    # Compute Geodesic Distance (longest shortest path along the skeleton)
    if len(skel_coords) > 1:
        # Compute pairwise distances along the skeleton
        dist_matrix = squareform(pdist(skel_coords))  
        max_distance = 0
        max_pair = None

        # Find the two most distant points in the skeleton
        for i in range(len(skel_coords)):
            for j in range(i + 1, len(skel_coords)):
                if dist_matrix[i, j] > max_distance:
                    max_distance = dist_matrix[i, j]
                    max_pair = (skel_coords[i], skel_coords[j])

        if max_pair:
            # Compute the geodesic distance using shortest path in skeleton
            cost_array = np.ones_like(binary, dtype=np.float32)  # Uniform cost map
            geodesic_distance, _ = route_through_array(
                cost_array, max_pair[0], max_pair[1], fully_connected=True
            )
            geodesic_distance = np.sum(geodesic_distance)  # Total path length
        else:
            geodesic_distance = 0
    else:
        geodesic_distance = 0  # No valid skeleton

    # Print results
    print(f"Contour {i+1}:")
    print(f"  Geodesic Distance: {geodesic_distance:.2f} pixels")
    print(f"  Ellipse Center: {ellipse[0]}, Axes: {ellipse[1]}, Angle: {ellipse[2]}\n")

# Display results
cv2.imshow("Ellipses on Contours", output_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
